## File to make /Coastal risk csv cf compliant and export to zarr

Notebook environment to migrate Coastal risk csv file to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
#%pip install tqdm

<IPython.core.display.Javascript object>

In [3]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math
from tqdm import tqdm 

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data"


# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)

<IPython.core.display.Javascript object>

In [4]:
template_df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\Arjen_Vector_Template.csv")
Hazard_map=template_df
Hazard_map = gpd.GeoDataFrame(
    Hazard_map, geometry=gpd.points_from_xy(Hazard_map.Intersect_lon, Hazard_map.Intersect_lat), crs="EPSG:4326"
)
Hazard_map=Hazard_map.drop(columns=["Start_lon","Start_lat","End_lon","End_lat","Intersect_lon","Intersect_lat"])
Hazard_map
IPCC=gpd.read_file(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\IPCC-WGI-reference-regions-v4.gpkg")
IPCC.head()
region_acronym= "CAR"
region_gdf=IPCC.loc[IPCC["Acronym"]==region_acronym,:]
# region_gdf=region_gdf.iloc[:,-1:]
region_gdf

Hazard_map=Hazard_map.sjoin(region_gdf)
Haz_map=Hazard_map
Hazard_map=Hazard_map.iloc[:,0:5]
Hazard_map["IPCC_REGION"]=Haz_map["Name"]
Hazard_map.index=range(0,len(Hazard_map.index))
Hazard_map

,transect_id,country_id,continent,country_name,geometry,IPCC_REGION
0,BOX_116_353_2,COL,South America,Colombia,POINT (-71.87980 12.25068),Caribbean
1,BOX_116_353_3,COL,South America,Colombia,POINT (-71.87698 12.24731),Caribbean
2,BOX_116_353_4,COL,South America,Colombia,POINT (-71.87563 12.24314),Caribbean
3,BOX_116_353_5,COL,South America,Colombia,POINT (-71.87523 12.23877),Caribbean
4,BOX_116_353_6,COL,South America,Colombia,POINT (-71.87483 12.23440),Caribbean
...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,POINT (-71.66819 17.96526),Caribbean
37505,BOX_139_002_62,DOM,North America,Dominican Republic,POINT (-71.66623 17.96142),Caribbean
37506,BOX_139_002_63,DOM,North America,Dominican Republic,POINT (-71.66458 17.95744),Caribbean
37507,BOX_139_002_64,DOM,North America,Dominican Republic,POINT (-71.66294 17.95347),Caribbean


<IPython.core.display.Javascript object>

In [5]:
# Project paths & files (manual input)
dataset_dir = pathlib.Path().joinpath(gca_data_dir,"Processed_Risk_Levels_PC_FC_Merged","Caribbean_Thesis_builtup_pop_ns_fix")
dataset_dir_path = dataset_dir.joinpath("PC_FC_RL_merged_Carib_original.nc")
CF_dir = dataset_dir.joinpath("CF")  # directory to save output CF check files
dataset_dir_path

WindowsPath('P:/11209197-018-global-coastal-atlas/MSc_students/ClenmarRowe/Data/Processed_Risk_Levels_PC_FC_Merged/Caribbean_Thesis_builtup_pop_ns_fix/PC_FC_RL_merged_Carib_original.nc')

<IPython.core.display.Javascript object>

In [6]:
PC_Risk_path=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_PC_builtup_pop_ns_fix" #Exposure and vuln from fixed builtup/pop/NS
FC_Risk_2050_high=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_FC\2050_High_emission"
FC_Risk_2050_Low=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_FC\2050_Low_emission"
FC_Risk_2100_High=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_FC\2100_High_emission"
FC_Risk_2100_Low=r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data_CVI_components\Caribbean Thesis_Results\Combination\Risk_Level_FC\2100_Low_emission"

<IPython.core.display.Javascript object>

In [7]:
all_paths=[PC_Risk_path,FC_Risk_2050_high,FC_Risk_2050_Low,FC_Risk_2100_High,FC_Risk_2100_Low]  

df_All_Risk=[]
for path in all_paths:
    files = os.listdir(path)

    # Filter only CSV files
    csv_files = [file for file in files if file.endswith('.csv')]


    
    # Iterate over each CSV file
    for csv_file in csv_files:
        file_path = os.path.join(path, csv_file)
        df = pd.read_csv(file_path)
        df_All_Risk.append(df)
df_All_Risk

[          transect_id  \
 0       BOX_116_353_2   
 1       BOX_116_353_3   
 2       BOX_116_353_4   
 3       BOX_116_353_5   
 4       BOX_116_353_6   
 ...               ...   
 37504  BOX_139_002_61   
 37505  BOX_139_002_62   
 37506  BOX_139_002_63   
 37507  BOX_139_002_64   
 37508  BOX_139_002_65   
 
        Cat3_Hurricane_Return_Period_PC_percentile - Hazard level  \
 0                                                      3           
 1                                                      3           
 2                                                      3           
 3                                                      3           
 4                                                      3           
 ...                                                  ...           
 37504                                                  3           
 37505                                                  3           
 37506                                                  3        

<IPython.core.display.Javascript object>

In [8]:
len(df_All_Risk)

18

<IPython.core.display.Javascript object>

In [9]:
Hazard_map

,transect_id,country_id,continent,country_name,geometry,IPCC_REGION
0,BOX_116_353_2,COL,South America,Colombia,POINT (-71.87980 12.25068),Caribbean
1,BOX_116_353_3,COL,South America,Colombia,POINT (-71.87698 12.24731),Caribbean
2,BOX_116_353_4,COL,South America,Colombia,POINT (-71.87563 12.24314),Caribbean
3,BOX_116_353_5,COL,South America,Colombia,POINT (-71.87523 12.23877),Caribbean
4,BOX_116_353_6,COL,South America,Colombia,POINT (-71.87483 12.23440),Caribbean
...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,POINT (-71.66819 17.96526),Caribbean
37505,BOX_139_002_62,DOM,North America,Dominican Republic,POINT (-71.66623 17.96142),Caribbean
37506,BOX_139_002_63,DOM,North America,Dominican Republic,POINT (-71.66458 17.95744),Caribbean
37507,BOX_139_002_64,DOM,North America,Dominican Republic,POINT (-71.66294 17.95347),Caribbean


<IPython.core.display.Javascript object>

In [10]:
Hazard_map.columns[1]

'country_id'

<IPython.core.display.Javascript object>

In [11]:
df_All_Risk[0].columns

Index(['transect_id',
       'Cat3_Hurricane_Return_Period_PC_percentile - Hazard level',
       'Percentage of Built Up Area_percentile - Exposure level_percentile',
       'LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile',
       'Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile',
       'Population Within 1km of the coastline_percentile - Exposure level_percentile',
       'Sediment Type_percentile - Exposure level_percentile',
       'Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile',
       'GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile',
       'Mean Years of Schooling_Percentile - Vulnerability level_percentile',
       'Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile',
       'World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile',
       'Risk Level', 'Risk Level Class', 'Uncertainty_Columns',
       'Uncerta

<IPython.core.display.Javascript object>

In [12]:
df_Exp_Vul=df_All_Risk[0].copy()
df_Exp_Vul["lon"]=df_Exp_Vul["geometry"].apply(lambda point_str: float(point_str.strip('POINT ()').split()[0]))
df_Exp_Vul["lat"]=df_Exp_Vul["geometry"].apply(lambda point_str: float(point_str.strip('POINT ()').split()[1]))
df_Exp_Vul=df_Exp_Vul.drop(columns=[df_All_Risk[0].columns[1],df_All_Risk[0].columns[12],df_All_Risk[0].columns[13],df_All_Risk[0].columns[-1],"geometry"])
df_Exp_Vul[Hazard_map.columns[1]]=Hazard_map[Hazard_map.columns[1]]
df_Exp_Vul[Hazard_map.columns[2]]=Hazard_map[Hazard_map.columns[2]]
df_Exp_Vul[Hazard_map.columns[3]]=Hazard_map[Hazard_map.columns[3]]
df_Exp_Vul[Hazard_map.columns[5]]=Hazard_map[Hazard_map.columns[5]]
df_Exp_Vul["Uncertainty_Columns"]=df_Exp_Vul["Uncertainty_Columns"].apply(lambda x:x.replace("[","").replace("]","").replace("β","B"))
df_Exp_Vul

,transect_id,Percentage of Built Up Area_percentile - Exposure level_percentile,LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile,Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile,Population Within 1km of the coastline_percentile - Exposure level_percentile,Sediment Type_percentile - Exposure level_percentile,Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile,GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile,Mean Years of Schooling_Percentile - Vulnerability level_percentile,Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile,World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile,Uncertainty_Columns,Uncertainty_Columns_length,Confindence %,lon,lat,country_id,continent,country_name,IPCC_REGION
0,BOX_116_353_2,1,4,4.0,1,4.0,3.0,4,4.0,5,3,,0,100.0,-71.879805,12.250684,COL,South America,Colombia,Caribbean
1,BOX_116_353_3,1,5,4.0,1,2.0,3.0,4,4.0,5,3,,0,100.0,-71.876981,12.247308,COL,South America,Colombia,Caribbean
2,BOX_116_353_4,1,5,4.0,1,4.0,3.0,4,4.0,5,3,,0,100.0,-71.875632,12.243144,COL,South America,Colombia,Caribbean
3,BOX_116_353_5,1,5,4.0,1,2.0,3.0,4,4.0,5,3,,0,100.0,-71.875233,12.238772,COL,South America,Colombia,Caribbean
4,BOX_116_353_6,1,5,4.0,1,2.0,3.0,4,4.0,5,3,,0,100.0,-71.874835,12.234400,COL,South America,Colombia,Caribbean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,1,4,3.0,2,4.0,5.0,3,3.0,4,5,,0,100.0,-71.668186,17.965259,DOM,North America,Dominican Republic,Caribbean
37505,BOX_139_002_62,1,4,3.0,2,4.0,5.0,3,3.0,4,5,,0,100.0,-71.666228,17.961418,DOM,North America,Dominican Republic,Caribbean
37506,BOX_139_002_63,1,5,3.0,2,4.0,5.0,3,3.0,4,5,,0,100.0,-71.664583,17.957442,DOM,North America,Dominican Republic,Caribbean
37507,BOX_139_002_64,1,5,3.0,2,4.0,5.0,3,3.0,4,5,,0,100.0,-71.662937,17.953466,DOM,North America,Dominican Republic,Caribbean


<IPython.core.display.Javascript object>

In [13]:
df_Haz_Exp_Vul=df_Exp_Vul.copy()
for idx,df in enumerate(df_All_Risk):
    PC=""
    if idx<4:
        PC=" PC"
    haz=df.columns[1].replace(" - Hazard level",PC)
    df_Haz_Exp_Vul[haz+" - Hazard Level"]=df.iloc[:,1]
    df_Haz_Exp_Vul[haz+" - Risk Level"]=df.iloc[:,12]
    df_Haz_Exp_Vul[haz+" - Risk Level Class"]=df.iloc[:,13]
    df_Haz_Exp_Vul[haz+" value"]=df.iloc[:,-1]
df_Haz_Exp_Vul.columns

Index(['transect_id',
       'Percentage of Built Up Area_percentile - Exposure level_percentile',
       'LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile',
       'Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile',
       'Population Within 1km of the coastline_percentile - Exposure level_percentile',
       'Sediment Type_percentile - Exposure level_percentile',
       'Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile',
       'GDP per Capita (3_yr_Average)_Percentile - Vulnerability level_percentile',
       'Mean Years of Schooling_Percentile - Vulnerability level_percentile',
       'Deaths from Recent Shocks (2004-2023)_Percentile - Vulnerability level_percentile',
       'World Protected Areas (Status and Governance)_Percentile - Vulnerability level_percentile',
       'Uncertainty_Columns', 'Uncertainty_Columns_length', 'Confindence %',
       'lon', 'lat', 'country_id', 'continent', 'country_name', 'IPCC_

<IPython.core.display.Javascript object>

In [14]:
df_Haz_Exp_Vul["Cat3_Hurricane_Return_Period_PC_percentile PC value"]

0        Cat3_Hurricane_Return_Period_PC_percentile 7.0...
1        Cat3_Hurricane_Return_Period_PC_percentile 7.0...
2        Cat3_Hurricane_Return_Period_PC_percentile 7.0...
3        Cat3_Hurricane_Return_Period_PC_percentile 7.0...
4        Cat3_Hurricane_Return_Period_PC_percentile 7.0...
                               ...                        
37504    Cat3_Hurricane_Return_Period_PC_percentile 6.3...
37505    Cat3_Hurricane_Return_Period_PC_percentile 6.3...
37506    Cat3_Hurricane_Return_Period_PC_percentile 6.3...
37507    Cat3_Hurricane_Return_Period_PC_percentile 6.3...
37508    Cat3_Hurricane_Return_Period_PC_percentile 6.3...
Name: Cat3_Hurricane_Return_Period_PC_percentile PC value, Length: 37509, dtype: object

<IPython.core.display.Javascript object>

In [15]:
df_Haz_Exp_Vul.to_csv(dataset_dir.joinpath("PC_FC_RL_merged_Carib.csv"),index=False)

<IPython.core.display.Javascript object>

In [16]:
ds=xr.Dataset.from_dataframe(df_Haz_Exp_Vul)
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                index: 37509)
Coordinates:
  * index                                                                                      (index) int64 ...
Data variables: (12/92)
    transect_id                                                                                (index) object ...
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (index) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (index) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (index) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (index) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (index) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (index) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (index) object ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (index) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (index) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (index) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (index) object ...

<IPython.core.display.Javascript object>

In [17]:

os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
# Write the xarray dataset to a netCDF file
ds.to_netcdf(dataset_dir_path)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [18]:
# open datasets
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                index: 37509)
Coordinates:
  * index                                                                                      (index) int64 ...
Data variables: (12/92)
    transect_id                                                                                (index) <U15 ...
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (index) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (index) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (index) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (index) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (index) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (index) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (index) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (index) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (index) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (index) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (index) <U30 ...

<IPython.core.display.Javascript object>

In [19]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile= dataset_dir_path, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [20]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\Processed_Risk_Levels_PC_FC_Merged\\Caribbean_Thesis_builtup_pop_ns_fix\\PC_FC_RL_merged_Carib_original.nc': {'warnings': '202', 'errors': '24'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [21]:
# open original datasets
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                index: 37509)
Coordinates:
  * index                                                                                      (index) int64 ...
Data variables: (12/92)
    transect_id                                                                                (index) <U15 ...
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (index) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (index) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (index) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (index) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (index) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (index) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (index) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (index) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (index) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (index) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (index) <U30 ...

<IPython.core.display.Javascript object>

In [22]:

# set some data variables to coordinates to *avoid duplication* of dimensions in later stage
ds = ds.set_coords(['transect_id', 'Uncertainty_Columns', 'Uncertainty_Columns_length', 'Confindence %', 'lat',"lon", 'country_name', 'continent','country_id',"IPCC_REGION"])
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                index: 37509)
Coordinates:
  * index                                                                                      (index) int64 ...
    transect_id                                                                                (index) <U15 ...
    Uncertainty_Columns                                                                        (index) <U225 ...
    Uncertainty_Columns_length                                                                 (index) int64 ...
    Confindence %                                                                              (index) float64 ...
    lon                                                                                        (index) float64 ...
    lat                                                                                        (index) float64 ...
    country_id                                                                                 (index) <U7 ...
    continent                                                                                  (index) <U13 ...
    country_name                                                                               (index) <U32 ...
    IPCC_REGION                                                                                (index) <U9 ...
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (index) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (index) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (index) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (index) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (index) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (index) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (index) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (index) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (index) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (index) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (index) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (index) <U30 ...

<IPython.core.display.Javascript object>

In [23]:
# rename or swap dimension names, the latter in case the name already exists as coordinate
ds = ds.rename_dims({"index": "nstations"})
ds


<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                nstations: 37509)
Coordinates:
  * index                                                                                      (nstations) int64 ...
    transect_id                                                                                (nstations) <U15 ...
    Uncertainty_Columns                                                                        (nstations) <U225 ...
    Uncertainty_Columns_length                                                                 (nstations) int64 ...
    Confindence %                                                                              (nstations) float64 ...
    lon                                                                                        (nstations) float64 ...
    lat                                                                                        (nstations) float64 ...
    country_id                                                                                 (nstations) <U7 ...
    continent                                                                                  (nstations) <U13 ...
    country_name                                                                               (nstations) <U32 ...
    IPCC_REGION                                                                                (nstations) <U9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (nstations) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (nstations) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (nstations) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (nstations) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (nstations) <U30 ...

<IPython.core.display.Javascript object>

In [24]:
ds=ds.drop_vars(["index"])
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                nstations: 37509)
Coordinates:
    transect_id                                                                                (nstations) <U15 ...
    Uncertainty_Columns                                                                        (nstations) <U225 ...
    Uncertainty_Columns_length                                                                 (nstations) int64 ...
    Confindence %                                                                              (nstations) float64 ...
    lon                                                                                        (nstations) float64 ...
    lat                                                                                        (nstations) float64 ...
    country_id                                                                                 (nstations) <U7 ...
    continent                                                                                  (nstations) <U13 ...
    country_name                                                                               (nstations) <U32 ...
    IPCC_REGION                                                                                (nstations) <U9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (nstations) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (nstations) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (nstations) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (nstations) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (nstations) <U30 ...

<IPython.core.display.Javascript object>

In [25]:
import json

# NetCDF attribute alterations
f_global = open(dataset_dir.joinpath("STOC_metadata.json"))
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                nstations: 37509)
Coordinates:
    transect_id                                                                                (nstations) <U15 ...
    Uncertainty_Columns                                                                        (nstations) <U225 ...
    Uncertainty_Columns_length                                                                 (nstations) int64 ...
    Confindence %                                                                              (nstations) float64 ...
    lon                                                                                        (nstations) float64 ...
    lat                                                                                        (nstations) float64 ...
    country_id                                                                                 (nstations) <U7 ...
    continent                                                                                  (nstations) <U13 ...
    country_name                                                                               (nstations) <U32 ...
    IPCC_REGION                                                                                (nstations) <U9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (nstations) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (nstations) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (nstations) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (nstations) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (nstations) <U30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [26]:
# rename variables, if necessary
ds = ds.rename_vars(
       {
        "country_name": "country" ,
       }
  )
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                nstations: 37509)
Coordinates:
    transect_id                                                                                (nstations) <U15 ...
    Uncertainty_Columns                                                                        (nstations) <U225 ...
    Uncertainty_Columns_length                                                                 (nstations) int64 ...
    Confindence %                                                                              (nstations) float64 ...
    lon                                                                                        (nstations) float64 ...
    lat                                                                                        (nstations) float64 ...
    country_id                                                                                 (nstations) <U7 ...
    continent                                                                                  (nstations) <U13 ...
    country                                                                                    (nstations) <U32 ...
    IPCC_REGION                                                                                (nstations) <U9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (nstations) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (nstations) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (nstations) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (nstations) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (nstations) <U30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [27]:
ds=ds.compute()
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                nstations: 37509)
Coordinates:
    transect_id                                                                                (nstations) <U15 ...
    Uncertainty_Columns                                                                        (nstations) <U225 ...
    Uncertainty_Columns_length                                                                 (nstations) int64 ...
    Confindence %                                                                              (nstations) float64 ...
    lon                                                                                        (nstations) float64 ...
    lat                                                                                        (nstations) float64 ...
    country_id                                                                                 (nstations) <U7 ...
    continent                                                                                  (nstations) <U13 ...
    country                                                                                    (nstations) <U32 ...
    IPCC_REGION                                                                                (nstations) <U9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (nstations) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (nstations) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (nstations) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (nstations) <U33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (nstations) <U30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [28]:
np.unique(ds["Uncertainty_Columns"].values)

array(['',
       "'Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile', 'Mean Years of Schooling_Percentile - Vulnerability level_percentile'",
       "'Mean Years of Schooling_Percentile - Vulnerability level_percentile'",
       "'Neashore Slope (tanB or m^-1)_percentile - Exposure level_percentile'",
       "'Neashore Slope (tanB or m^-1)_percentile - Exposure level_percentile', 'Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile', 'Mean Years of Schooling_Percentile - Vulnerability level_percentile'",
       "'Neashore Slope (tanB or m^-1)_percentile - Exposure level_percentile', 'Mean Years of Schooling_Percentile - Vulnerability level_percentile'",
       "'Sediment Type_percentile - Exposure level_percentile'"],
      dtype='<U225')

<IPython.core.display.Javascript object>

In [29]:
# change dtypes from unsigned to signed
object_vars = list(ds.variables)
for i in object_vars:
    # if isinstance(ds[i].values[0],str) and (i not in ['Uncertainty_Columns'] ) :
    if isinstance(ds[i].values[0],str)  :
        ds[i] = ds[i].astype('S')
ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                nstations: 37509)
Coordinates:
    transect_id                                                                                (nstations) |S15 ...
    Uncertainty_Columns                                                                        (nstations) |S225 ...
    Uncertainty_Columns_length                                                                 (nstations) int64 ...
    Confindence %                                                                              (nstations) float64 ...
    lon                                                                                        (nstations) float64 ...
    lat                                                                                        (nstations) float64 ...
    country_id                                                                                 (nstations) |S7 ...
    continent                                                                                  (nstations) |S13 ...
    country                                                                                    (nstations) |S32 ...
    IPCC_REGION                                                                                (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (nstations) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (nstations) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (nstations) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [30]:
i

'EWH_100yrRP_2100_RCP_45 value'

<IPython.core.display.Javascript object>

In [31]:
# add or change certain variable / coordinate attributes
### dataset attributes is a dictionary of dictionaries
dataset_attributes = {
    "lon": {"standard_name": "longitude", "long_name": "longitude", "units": "degrees_east"},
    "lat": {"standard_name": "latitude", "long_name": "latitude", "units": "degrees_north"},
    "transect_id": { "long_name": "Transect Identity", "units": "1"},
    "continent": { "long_name": "Continent", "units": "1"},
    "country": { "long_name": "Country", "units": "1"},
    "country_id": { "long_name": "Country Identification", "units": "1"},
    "IPCC_REGION": { "long_name": "IPCC Region", "units": "1"}
}  # specify custom (CF convention) attributes

 # add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        ds[k].attrs = dataset_attributes[k]
    except:
        continue


ds

<xarray.Dataset>
Dimensions:                                                                                    (
                                                                                                nstations: 37509)
Coordinates:
    transect_id                                                                                (nstations) |S15 ...
    Uncertainty_Columns                                                                        (nstations) |S225 ...
    Uncertainty_Columns_length                                                                 (nstations) int64 ...
    Confindence %                                                                              (nstations) float64 ...
    lon                                                                                        (nstations) float64 ...
    lat                                                                                        (nstations) float64 ...
    country_id                                                                                 (nstations) |S7 ...
    continent                                                                                  (nstations) |S13 ...
    country                                                                                    (nstations) |S32 ...
    IPCC_REGION                                                                                (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area_percentile - Exposure level_percentile                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation_percentile - Exposure level_percentile                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1)_percentile - Exposure level_percentile                       (nstations) float64 ...
    Population Within 1km of the coastline_percentile - Exposure level_percentile              (nstations) int64 ...
    Sediment Type_percentile - Exposure level_percentile                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average)_Percentile - Vulnerability level_percentile           (nstations) float64 ...
    ...                                                                                         ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                                             (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                                          (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                                                     (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                                       (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                                              (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [32]:
#Remove the percentile from the variable names

new_names = {var: var.replace('_percentile', '') for var in ds.variables if '_percentile' in var}
ds = ds.rename(new_names)
new_names = {var: var.replace('_Percentile', '') for var in ds.variables if '_Percentile' in var}
ds = ds.rename(new_names)
new_names = {var: var.replace('Exposure level ', 'Exposure level') for var in ds.variables if 'Exposure level ' in var}
ds = ds.rename(new_names)
new_names = {var: var.replace('Exposure level ', 'Exposure level') for var in ds.variables if 'Exposure level ' in var}
ds = ds.rename(new_names)

ds

<xarray.Dataset>
Dimensions:                                                              (
                                                                          nstations: 37509)
Coordinates:
    transect_id                                                          (nstations) |S15 ...
    Uncertainty_Columns                                                  (nstations) |S225 ...
    Uncertainty_Columns_length                                           (nstations) int64 ...
    Confindence %                                                        (nstations) float64 ...
    lon                                                                  (nstations) float64 ...
    lat                                                                  (nstations) float64 ...
    country_id                                                           (nstations) |S7 ...
    continent                                                            (nstations) |S13 ...
    country                                                              (nstations) |S32 ...
    IPCC_REGION                                                          (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area - Exposure level                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation - Exposure level                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1) - Exposure level                       (nstations) float64 ...
    Population Within 1km of the coastline - Exposure level              (nstations) int64 ...
    Sediment Type - Exposure level                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average) - Vulnerability level           (nstations) float64 ...
    ...                                                                   ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                       (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                    (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                               (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                           (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                        (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [33]:
dataset_attributes = {
    "Uncertainty_Columns": {"long_name": "Uncertainty Columns"},
    "Uncertainty_Columns_length": {"long_name": "Uncertainty Columns Length"},
    "Confindence %": {"long_name": "Confindence Percentage"},
    "Percentage of Built Up Area - Exposure level": {"long_name": "Percentage of Built Up Area - Exposure level"},
    "LECZ Area÷LECZ mean elevation - Exposure level": {"long_name": "LECZ Area÷LECZ mean elevation - Exposure level"},
    "Neashore Slope (tanβ or m^-1) - Exposure level": {"long_name": "Neashore Slope (tanβ or m^-1) - Exposure level"},
    "Population Within 1km of the coastline - Exposure level": {"long_name": "Population Within 1km of the coastline - Exposure level"},
    "Sediment Type - Exposure level": {"long_name": "Sediment Type - Exposure level"},
    "Infant Mortality Rate (3_yr_Average) - Vulnerability level": {"long_name": "Infant Mortality Rate (3_yr_Average) - Vulnerability level"},
    "GDP per Capita (3_yr_Average) - Vulnerability level": {"long_name": "GDP per Capita (3_yr_Average) - Vulnerability level"},
    "Mean Years of Schooling - Vulnerability level": {"long_name": "Mean Years of Schooling - Vulnerability level"},
    "Deaths from Recent Shocks (2004-2023) - Vulnerability level": {"long_name": "Deaths from Recent Shocks (2004-2023) - Vulnerability level"},
    "World Protected Areas (Status and Governance) - Vulnerability level": {"long_name": "World Protected Areas (Status and Governance) - Vulnerability level"},
    "ESL_100yrRP PC - Hazard Level": {"long_name": "ESL_100yrRP PC - Hazard Level"},
    "ESL_100yrRP PC - Risk Level": {"long_name": "ESL_100yrRP PC - Risk Level"},
    "ESL_100yrRP PC - Risk Level Class": {"long_name": "ESL_100yrRP PC - Risk Level Class"},
    "ESL_100yrRP PC value": {"long_name": "ESL_100yrRP PC value"},
    "EWH_100yrRP PC - Hazard Level": {"long_name": "EWH_100yrRP PC - Hazard Level"},
    "EWH_100yrRP PC - Risk Level": {"long_name": "EWH_100yrRP PC - Risk Level"},
    "EWH_100yrRP PC - Risk Level Class": {"long_name": "EWH_100yrRP PC - Risk Level Class"},
    "EWH_100yrRP PC value": {"long_name": "EWH_100yrRP PC value"},
    "Land_Subsidence_2010 PC - Hazard Level": {"long_name": "Land Subsidence 2010 PC - Hazard Level"},
    "Land_Subsidence_2010 PC - Risk Level": {"long_name": "Land Subsidence 2010 PC - Risk Level"},
    "Land_Subsidence_2010 PC - Risk Level Class": {"long_name": "Land Subsidence 2010 PC - Risk Level Class"},
    "Land_Subsidence_2010 PC value": {"long_name": "Land Subsidence 2010 PC value"},
    "Shoreline_Change_Rate_PC - Hazard Level": {"long_name": "Shoreline Change Rate PC - Hazard Level"},
    "Shoreline_Change_Rate_PC - Risk Level": {"long_name": "Shoreline Change Rate PC - Risk Level"},
    "Shoreline_Change_Rate_PC - Risk Level Class": {"long_name": "Shoreline Change Rate PC - Risk Level Class"},
    "Shoreline_Change_Rate_PC value": {"long_name": "Shoreline Change Rate PC value"},
    "Change_rate_SSP5_85_2050 - Hazard Level": {"long_name": "Change rate SSP5 85 2050 - Hazard Level"},
    "Change_rate_SSP5_85_2050 - Risk Level": {"long_name": "Change rate SSP5 85 2050 - Risk Level"},
    "Change_rate_SSP5_85_2050 - Risk Level Class": {"long_name": "Change rate SSP5 85 2050 - Risk Level Class"},
    "Change_rate_SSP5_85_2050 value": {"long_name": "Change rate SSP5 85 2050 value"},
    "ESL_100yrRP_8.5_2050 - Hazard Level": {"long_name": "ESL 100yrRP 8.5 2050 - Hazard Level"},
    "ESL_100yrRP_8.5_2050 - Risk Level": {"long_name": "ESL 100yrRP 8.5 2050 - Risk Level"},
    "ESL_100yrRP_8.5_2050 - Risk Level Class": {"long_name": "ESL 100yrRP 8.5 2050 - Risk Level Class"},
    "ESL_100yrRP_8.5_2050 value": {"long_name": "ESL 100yrRP 8.5 2050 value"},
    "EWH_100yrRP_2050_RCP_85 - Hazard Level": {"long_name": "EWH 100yrRP 2050 RCP 85 - Hazard Level"},
    "EWH_100yrRP_2050_RCP_85 - Risk Level": {"long_name": "EWH 100yrRP 2050 RCP 85 - Risk Level"},
    "EWH_100yrRP_2050_RCP_85 - Risk Level Class": {"long_name": "EWH 100yrRP 2050 RCP 85 - Risk Level Class"},
    "EWH_100yrRP_2050_RCP_85 value": {"long_name": "EWH 100yrRP 2050 RCP 85 value"},
    "Land_Subsidence_2040_high - Hazard Level": {"long_name": "Land Subsidence 2040 high - Hazard Level"},
    "Land_Subsidence_2040_high - Risk Level": {"long_name": "Land Subsidence 2040 high - Risk Level"},
    "Land_Subsidence_2040_high - Risk Level Class": {"long_name": "Land Subsidence 2040 high - Risk Level Class"},
    "Land_Subsidence_2040_high value": {"long_name": "Land Subsidence 2040 high value"},
    "Change_rate_SSP1_45_2050 - Hazard Level": {"long_name": "Change rate SSP1 45 2050 - Hazard Level"},
    "Change_rate_SSP1_45_2050 - Risk Level": {"long_name": "Change rate SSP1 45 2050 - Risk Level"},
    "Change_rate_SSP1_45_2050 - Risk Level Class": {"long_name": "Change rate SSP1 45 2050 - Risk Level Class"},
    "Change_rate_SSP1_45_2050 value": {"long_name": "Change rate SSP1 45 2050 value"},
    "ESL_100yrRP_4.5_2050 - Hazard Level": {"long_name": "ESL 100yrRP 4.5 2050 - Hazard Level"},
    "ESL_100yrRP_4.5_2050 - Risk Level": {"long_name": "ESL 100yrRP 4.5 2050 - Risk Level"},
    "ESL_100yrRP_4.5_2050 - Risk Level Class": {"long_name": "ESL 100yrRP 4.5 2050 - Risk Level Class"},
    "ESL_100yrRP_4.5_2050 value": {"long_name": "ESL 100yrRP 4.5 2050 value"},
    "EWH_100yrRP_2050_RCP_45 - Hazard Level": {"long_name": "EWH 100yrRP 2050 RCP 45 - Hazard Level"},
    "EWH_100yrRP_2050_RCP_45 - Risk Level": {"long_name": "EWH 100yrRP 2050 RCP 45 - Risk Level"},
    "EWH_100yrRP_2050_RCP_45 - Risk Level Class": {"long_name": "EWH 100yrRP 2050 RCP 45 - Risk Level Class"},
    "EWH_100yrRP_2050_RCP_45 value": {"long_name": "EWH 100yrRP 2050 RCP 45 value"},
    "Change_rate_SSP5_85_2100 - Hazard Level": {"long_name": "Change rate SSP5 85 2100 - Hazard Level"},
    "Change_rate_SSP5_85_2100 - Risk Level": {"long_name": "Change rate SSP5 85 2100 - Risk Level"},
    "Change_rate_SSP5_85_2100 - Risk Level Class": {"long_name": "Change rate SSP5 85 2100 - Risk Level Class"},
    "Change_rate_SSP5_85_2100 value": {"long_name": "Change rate SSP5 85 2100 value"},
    "ESL_100yrRP_8.5_2100 - Hazard Level": {"long_name": "ESL 100yrRP 8.5 2100 - Hazard Level"},
    "ESL_100yrRP_8.5_2100 - Risk Level": {"long_name": "ESL 100yrRP 8.5 2100 - Risk Level"},
    "ESL_100yrRP_8.5_2100 - Risk Level Class": {"long_name": "ESL 100yrRP 8.5 2100 - Risk Level Class"},
    "ESL_100yrRP_8.5_2100 value": {"long_name": "ESL 100yrRP 8.5 2100 value"},
    "EWH_100yrRP_2100_RCP_45 - Hazard Level": {"long_name": "EWH 100yrRP 2100 RCP 45 - Hazard Level"},
    "EWH_100yrRP_2100_RCP_45 - Risk Level": {"long_name": "EWH 100yrRP 2100 RCP 45 - Risk Level"},
    "EWH_100yrRP_2100_RCP_45 - Risk Level Class": {"long_name": "EWH 100yrRP 2100 RCP 45 - Risk Level Class"},
    "EWH_100yrRP_2100_RCP_45 value": {"long_name": "EWH 100yrRP 2100 RCP 45 value"},
    "Change_rate_SSP1_45_2100 - Hazard Level": {"long_name": "Change rate SSP1 45 2100 - Hazard Level"},
    "Change_rate_SSP1_45_2100 - Risk Level": {"long_name": "Change rate SSP1 45 2100 - Risk Level"},
    "Change_rate_SSP1_45_2100 - Risk Level Class": {"long_name": "Change rate SSP1 45 2100 - Risk Level Class"},
    "Change_rate_SSP1_45_2100 value": {"long_name": "Change rate SSP1 45 2100 value"},
    "ESL_RCP4.5_Time_Horizon2100 - Hazard Level": {"long_name": "ESL RCP4.5 Time Horizon2100 - Hazard Level"},
    "ESL_RCP4.5_Time_Horizon2100 - Risk Level": {"long_name": "ESL RCP4.5 Time Horizon2100 - Risk Level"},
    "ESL_RCP4.5_Time_Horizon2100 - Risk Level Class": {"long_name": "ESL RCP4.5 Time Horizon2100 - Risk Level Class"},
    "ESL_RCP4.5_Time_Horizon2100 value": {"long_name": "ESL RCP4.5 Time Horizon2100 value"},
    "EWH_100yrRP_2100_RCP_85 - Hazard Level": {"long_name": "EWH 100yrRP 2100 RCP 85 - Hazard Level"},
    "EWH_100yrRP_2100_RCP_85 - Risk Level": {"long_name": "EWH 100yrRP 2100 RCP 85 - Risk Level"},
    "EWH_100yrRP_2100_RCP_85 - Risk Level Class": {"long_name": "EWH 100yrRP 2100 RCP 85 - Risk Level Class"},
    "EWH_100yrRP_2100_RCP_85 value": {"long_name": "EWH 100yrRP 2100 RCP 85 value"},
    "Cat3_Hurricane_Return_Period_PC PC - Hazard Level": {"long_name": "Cat3 Hurricane Return Period PC - Hazard Level"},
    "Cat3_Hurricane_Return_Period_PC PC - Risk Level": {"long_name": "Cat3 Hurricane Return Period PC - Risk Level"},
    "Cat3_Hurricane_Return_Period_PC PC - Risk Level Class": {"long_name": "Cat3 Hurricane Return Period PC - Risk Level Class"},
    "Cat3_Hurricane_Return_Period_PC PC value": {"long_name": "Cat3 Hurricane Return Period PC value"},
    "Cat3_TC_Return_Period_FC_2050_4_5 - Hazard Level": {"long_name": "Cat3 TC Return Period FC 2050 45 - Hazard Level"},
    "Cat3_TC_Return_Period_FC_2050_4_5 - Risk Level": {"long_name": "Cat3 TC Return Period FC 2050 45 - Risk Level"},
    "Cat3_TC_Return_Period_FC_2050_4_5 - Risk Level Class": {"long_name": "Cat3 TC Return Period FC 2050 45 - Risk Level Class"},
    "Cat3_TC_Return_Period_FC_2050_4_5 value": {"long_name": "Cat3 TC Return Period FC 2050 45 value"},
    "Cat3_TC_Return_Period_FC_2050_8_5 - Hazard Level": {"long_name": "Cat3 TC Return Period FC 2050 85 - Hazard Level"},
    "Cat3_TC_Return_Period_FC_2050_8_5 - Risk Level": {"long_name": "Cat3 TC Return Period FC 2050 85 - Risk Level"},
    "Cat3_TC_Return_Period_FC_2050_8_5 - Risk Level Class": {"long_name": "Cat3 TC Return Period FC 2050 85 - Risk Level Class"},
    "Cat3_TC_Return_Period_FC_2050_8_5 value": {"long_name": "Cat3 TC Return Period FC 2050 85 value"},
    "Cat3_TC_Return_Period_FC_2100_4_5 - Hazard Level": {"long_name": "Cat3 TC Return Period FC 2100 45 - Hazard Level"},
    "Cat3_TC_Return_Period_FC_2100_4_5 - Risk Level": {"long_name": "Cat3 TC Return Period FC 2100 45 - Risk Level"},
    "Cat3_TC_Return_Period_FC_2100_4_5 - Risk Level Class": {"long_name": "Cat3 TC Return Period FC 2100 45 - Risk Level Class"},
    "Cat3_TC_Return_Period_FC_2100_4_5 value": {"long_name": "Cat3 TC Return Period FC 2100 45 value"},
    "Cat3_TC_Return_Period_FC_2100_8_5 - Hazard Level": {"long_name": "Cat3 TC Return Period FC 2100 85 - Hazard Level"},
    "Cat3_TC_Return_Period_FC_2100_8_5 - Risk Level": {"long_name": "Cat3 TC Return Period FC 2100 85 - Risk Level"},
    "Cat3_TC_Return_Period_FC_2100_8_5 - Risk Level Class": {"long_name": "Cat3 TC Return Period FC 2100 85 - Risk Level Class"},
    "Cat3_TC_Return_Period_FC_2100_8_5 value": {"long_name": "Cat3 TC Return Period FC 2100 85 value"},
}

# Add or update attributes in the xarray Dataset ds
for var in dataset_attributes:
    if var in ds:
        ds[var].attrs.update(dataset_attributes[var])
ds

<xarray.Dataset>
Dimensions:                                                              (
                                                                          nstations: 37509)
Coordinates:
    transect_id                                                          (nstations) |S15 ...
    Uncertainty_Columns                                                  (nstations) |S225 ...
    Uncertainty_Columns_length                                           (nstations) int64 ...
    Confindence %                                                        (nstations) float64 ...
    lon                                                                  (nstations) float64 ...
    lat                                                                  (nstations) float64 ...
    country_id                                                           (nstations) |S7 ...
    continent                                                            (nstations) |S13 ...
    country                                                              (nstations) |S32 ...
    IPCC_REGION                                                          (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area - Exposure level                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation - Exposure level                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1) - Exposure level                       (nstations) float64 ...
    Population Within 1km of the coastline - Exposure level              (nstations) int64 ...
    Sediment Type - Exposure level                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average) - Vulnerability level           (nstations) float64 ...
    ...                                                                   ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                       (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                    (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                               (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                           (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                        (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [34]:
# Update dataset_attributes with suitable standard, long names, and units for all attributes
for attribute in dataset_attributes:
    units = "1"  # Default units
    
    if "ESL" in attribute:
        units = "m"
    elif "EWH" in attribute:
        units = "m"
    elif "Change_rate" in attribute:
        units = "m/yr"
    elif "Land_Subsidence_" in attribute:
        units = "1"
    elif "Cat3" in attribute:
        units = "year"

    
    dataset_attributes[attribute]["units"] = units

# Add or update attributes in the xarray Dataset ds
for var in dataset_attributes:
    if var in ds:
        ds[var].attrs.update(dataset_attributes[var])
ds

<xarray.Dataset>
Dimensions:                                                              (
                                                                          nstations: 37509)
Coordinates:
    transect_id                                                          (nstations) |S15 ...
    Uncertainty_Columns                                                  (nstations) |S225 ...
    Uncertainty_Columns_length                                           (nstations) int64 ...
    Confindence %                                                        (nstations) float64 ...
    lon                                                                  (nstations) float64 ...
    lat                                                                  (nstations) float64 ...
    country_id                                                           (nstations) |S7 ...
    continent                                                            (nstations) |S13 ...
    country                                                              (nstations) |S32 ...
    IPCC_REGION                                                          (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage of Built Up Area - Exposure level                         (nstations) int64 ...
    LECZ Area÷LECZ mean elevation - Exposure level                       (nstations) int64 ...
    Neashore Slope (tanβ or m^-1) - Exposure level                       (nstations) float64 ...
    Population Within 1km of the coastline - Exposure level              (nstations) int64 ...
    Sediment Type - Exposure level                                       (nstations) float64 ...
    Infant Mortality Rate (3_yr_Average) - Vulnerability level           (nstations) float64 ...
    ...                                                                   ...
    ESL_RCP4.5_Time_Horizon2100 - Risk Level Class                       (nstations) float64 ...
    ESL_RCP4.5_Time_Horizon2100 value                                    (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45 - Hazard Level                               (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level                                 (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 - Risk Level Class                           (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45 value                                        (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [35]:
renamed_vars = {var: var.replace("-", "_").replace(".", "_").replace(" ","_").replace("(","").replace(")","").replace("%","perc").replace("β","B").replace("^","").replace("÷","_") for var in ds.variables}
ds = ds.rename(renamed_vars)
ds

<xarray.Dataset>
Dimensions:                                                            (
                                                                        nstations: 37509)
Coordinates:
    transect_id                                                        (nstations) |S15 ...
    Uncertainty_Columns                                                (nstations) |S225 ...
    Uncertainty_Columns_length                                         (nstations) int64 ...
    Confindence_perc                                                   (nstations) float64 ...
    lon                                                                (nstations) float64 ...
    lat                                                                (nstations) float64 ...
    country_id                                                         (nstations) |S7 ...
    continent                                                          (nstations) |S13 ...
    country                                                            (nstations) |S32 ...
    IPCC_REGION                                                        (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage_of_Built_Up_Area___Exposure_level                       (nstations) int64 ...
    LECZ_Area_LECZ_mean_elevation___Exposure_level                     (nstations) int64 ...
    Neashore_Slope_tanB_or_m_1___Exposure_level                        (nstations) float64 ...
    Population_Within_1km_of_the_coastline___Exposure_level            (nstations) int64 ...
    Sediment_Type___Exposure_level                                     (nstations) float64 ...
    Infant_Mortality_Rate_3_yr_Average___Vulnerability_level           (nstations) float64 ...
    ...                                                                 ...
    ESL_RCP4_5_Time_Horizon2100___Risk_Level_Class                     (nstations) float64 ...
    ESL_RCP4_5_Time_Horizon2100_value                                  (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45___Hazard_Level                             (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45___Risk_Level                               (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45___Risk_Level_Class                         (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45_value                                      (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [36]:
# set some data variables to coordinates to *avoid duplication* of dimensions in later stage
ds = ds.set_coords(['transect_id', 'Uncertainty_Columns', 'Uncertainty_Columns_length', 'Confindence_perc', 'lat',"lon", 'country', 'continent','country_id'])
ds

<xarray.Dataset>
Dimensions:                                                            (
                                                                        nstations: 37509)
Coordinates:
    transect_id                                                        (nstations) |S15 ...
    Uncertainty_Columns                                                (nstations) |S225 ...
    Uncertainty_Columns_length                                         (nstations) int64 ...
    Confindence_perc                                                   (nstations) float64 ...
    lon                                                                (nstations) float64 ...
    lat                                                                (nstations) float64 ...
    country_id                                                         (nstations) |S7 ...
    continent                                                          (nstations) |S13 ...
    country                                                            (nstations) |S32 ...
    IPCC_REGION                                                        (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage_of_Built_Up_Area___Exposure_level                       (nstations) int64 ...
    LECZ_Area_LECZ_mean_elevation___Exposure_level                     (nstations) int64 ...
    Neashore_Slope_tanB_or_m_1___Exposure_level                        (nstations) float64 ...
    Population_Within_1km_of_the_coastline___Exposure_level            (nstations) int64 ...
    Sediment_Type___Exposure_level                                     (nstations) float64 ...
    Infant_Mortality_Rate_3_yr_Average___Vulnerability_level           (nstations) float64 ...
    ...                                                                 ...
    ESL_RCP4_5_Time_Horizon2100___Risk_Level_Class                     (nstations) float64 ...
    ESL_RCP4_5_Time_Horizon2100_value                                  (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45___Hazard_Level                             (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45___Risk_Level                               (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45___Risk_Level_Class                         (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45_value                                      (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [37]:
# Write the xarray dataset to a netCDF file
#Compliant netcdf
dataset_dir_path_CF=str(dataset_dir_path).replace("original","final")


ds.to_netcdf(path=dataset_dir_path_CF)

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [38]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path_CF)

# check original dataset
ds

<xarray.Dataset>
Dimensions:                                                            (
                                                                        nstations: 37509)
Coordinates:
    transect_id                                                        (nstations) |S15 ...
    Uncertainty_Columns                                                (nstations) |S225 ...
    Uncertainty_Columns_length                                         (nstations) int64 ...
    Confindence_perc                                                   (nstations) float64 ...
    lon                                                                (nstations) float64 ...
    lat                                                                (nstations) float64 ...
    country_id                                                         (nstations) |S7 ...
    continent                                                          (nstations) |S13 ...
    country                                                            (nstations) |S32 ...
    IPCC_REGION                                                        (nstations) |S9 ...
Dimensions without coordinates: nstations
Data variables: (12/82)
    Percentage_of_Built_Up_Area___Exposure_level                       (nstations) int64 ...
    LECZ_Area_LECZ_mean_elevation___Exposure_level                     (nstations) int64 ...
    Neashore_Slope_tanB_or_m_1___Exposure_level                        (nstations) float64 ...
    Population_Within_1km_of_the_coastline___Exposure_level            (nstations) int64 ...
    Sediment_Type___Exposure_level                                     (nstations) float64 ...
    Infant_Mortality_Rate_3_yr_Average___Vulnerability_level           (nstations) float64 ...
    ...                                                                 ...
    ESL_RCP4_5_Time_Horizon2100___Risk_Level_Class                     (nstations) float64 ...
    ESL_RCP4_5_Time_Horizon2100_value                                  (nstations) |S33 ...
    EWH_100yrRP_2100_RCP_45___Hazard_Level                             (nstations) int64 ...
    EWH_100yrRP_2100_RCP_45___Risk_Level                               (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45___Risk_Level_Class                         (nstations) float64 ...
    EWH_100yrRP_2100_RCP_45_value                                      (nstations) |S30 ...
Attributes: (12/22)
    TITLE:               State of the Coast- Present and Future risk levels (...
    TITLE_ABBREVIATION:  SOTC- Risk Levels
    DESCRIPTION:         The dataset contains qualitative shoreline parallel ...
    SHORT_DESCRIPTION:   This dataset provides a comprehensive analysis of co...
    INDICATORS_USED:     {"Cat 3 Tropical cyclones in the Caribbean": {"indic...
    INSTITUTION:         Deltares
    ...                  ...
    DOI:                 
    LONG_NAME:           State of the Coast (SOTC) - Caribbean
    UNITS:               meters
    COMMENT:             The dataset, titled 'State of the Coast- Present and...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [39]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [40]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\Processed_Risk_Levels_PC_FC_Merged\\Caribbean_Thesis_builtup_pop_ns_fix\\PC_FC_RL_merged_Carib_final.nc': {'warnings': '12', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [41]:
# # export to zarr in write mode (to overwrite if exists)
ds.to_zarr(str(dataset_dir_path).replace("original","final").replace(".nc", ".zarr"), mode="w")

<IPython.core.display.Javascript object>